#Introduction to Numba

##What is Numba?
Numba is a **just-in-time**, **type-specializing**, **function compiler** for accelerating **numerically-focused** Python. That's a long list, so let's break down those terms:

+ **function compiler**: Numba compiles Python functions, not entire applications, and not parts of functions. Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function.
+ **type-specializing**: Numba speeds up your function by generating a specialized implementation for the specific data types you are using. Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow. In practice, you only will call a function with a small number of argument types, so Numba will generate a fast implementation for each set of types.
+ **just-in-time**: Numba translates functions when they are first called. This ensures the compiler knows what argument types you will be using. This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application
+ **numerically-focused**: Currently, Numba is focused on numerical data types, like int, float, and complex. There is very limited string processing support, and many string use cases are not going to work well on the GPU. To get best results with Numba, you will likely be using NumPy arrays.

###First Steps
Let's write our first Numba function and compile it for the CPU. The Numba compiler is typically enabled by applying a decorator to a Python function. Decorators are functions that transform Python functions. Here we will use the CPU compilation decorator:

In [0]:
from numba import jit
import math
import numpy as np

In [0]:
@jit
def hypot(x, y):
  # Implementation from https://en.wikipedia.org/wiki/Hypot
  x = abs(x);
  y = abs(y);
  t = min(x, y);
  x = max(x, y);
  t = t / x;
  return x * math.sqrt(1+t*t)

Let's try out our hypotenuse calculation:

In [7]:
hypot(3.0,4.0)

5.0

The first time we call `hypot`, the compiler is triggered and compiles a machine code implementation for float inputs. Numba also saves the original Python implementation of the function in the `.py_func` attribute, so we can call the original Python code to make sure we get the same answer:

In [6]:
hypot.py_func(3.0,4.0)

5.0

###Benchmarking
An important part of using Numba is measuring the performance of your new code. Let's see if we actually sped anything up. The easiest way to do this in the Jupyter notebook is to use the `%timeit` magic function. Let's first measure the speed of the original Python:

In [9]:
%timeit hypot.py_func(3.0,4.0)

The slowest run took 16.63 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 867 ns per loop


The `%timeit` magic runs the statement many times to get an accurate estimate of the run time. It also returns the best time by default, which is useful to reduce the probability that random background events affect your measurement. The best of 3 approach also ensures that the compilation time on the first call doesn't skew the results:

In [10]:
%timeit hypot(3.0,4.0)

The slowest run took 31.32 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 273 ns per loop


Numba did a pretty good job with this function. It's more than 4x faster than the pure Python version.
Of course, the hypot function is already present in the Python module:

In [11]:
%timeit math.hypot(3.0,4.0)

The slowest run took 135.87 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 170 ns per loop


Python's built-in is even faster than Numba! This is because Numba does introduce some overhead to each function call that is larger than the function call overhead of Python itself. Extremely fast functions (like the above one) will be hurt by this.
(However, if you call one Numba function from another one, there is very little function overhead, sometimes even zero if the compiler inlines the function into the other one.)